# Data Quality & Validation Report

## Purpose
This notebook validates the reliability of the sales dataset
after ingestion and type enforcement.

It provides transparent checks that demonstrate:
- Data completeness
- Business rule consistency
- Revenue integrity
- Temporal coverage

This report is intended for stakeholders, reviewers, and clients.


## 1. Load Raw Data

The raw dataset is loaded without transformation.
All validation compares raw values to derived or expected behavior.


In [27]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/raw/sales_data.csv")


## 2. Dataset Overview

We first validate basic structural properties:
- Row count
- Column count
- Memory footprint


In [28]:
df.shape


(185950, 13)

In [29]:
df.columns

Index(['Unnamed: 0', 'Order ID', 'Product Category', 'Product',
       'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address',
       'Month', 'Sales', 'City', 'Hour', 'Time of Day'],
      dtype='object')

In [30]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        185950 non-null  int64  
 1   Order ID          185950 non-null  int64  
 2   Product Category  185950 non-null  object 
 3   Product           185950 non-null  object 
 4   Quantity Ordered  185950 non-null  int64  
 5   Price Each        185950 non-null  float64
 6   Order Date        185950 non-null  object 
 7   Purchase Address  185950 non-null  object 
 8   Month             185950 non-null  int64  
 9   Sales             185950 non-null  float64
 10  City              185950 non-null  object 
 11  Hour              185950 non-null  int64  
 12  Time of Day       185950 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 18.4+ MB


In [31]:

df.describe(include="all")


,Unnamed: 0,Order ID,Product Category,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour,Time of Day
count,185950.000000,185950.000000,185950,185950,185950.000000,185950.000000,185950,185950,185950.000000,185950.000000,185950,185950.000000,185950
unique,NaN,NaN,8,19,NaN,NaN,142395,140787,NaN,NaN,9,NaN,4
top,NaN,NaN,Audio Devices,USB-C Charging Cable,NaN,NaN,15-12-2019 20:16,"193 Forest St, San Francisco, CA 94016",NaN,NaN,San Francisco,NaN,Afternoon
freq,NaN,NaN,47756,21903,NaN,NaN,8,9,NaN,NaN,44732,NaN,67158
mean,8340.388475,230417.569379,NaN,NaN,1.124383,184.399735,NaN,NaN,7.059140,185.490917,NaN,14.413305,NaN
std,5450.554093,51512.737110,NaN,NaN,0.442793,332.731330,NaN,NaN,3.502996,332.919771,NaN,5.423416,NaN
min,0.000000,141234.000000,NaN,NaN,1.000000,2.990000,NaN,NaN,1.000000,2.990000,NaN,0.000000,NaN
25%,3894.000000,185831.250000,NaN,NaN,1.000000,11.950000,NaN,NaN,4.000000,11.950000,NaN,11.000000,NaN
50%,7786.000000,230367.500000,NaN,NaN,1.000000,14.950000,NaN,NaN,7.000000,14.950000,NaN,15.000000,NaN
75%,11872.000000,275035.750000,NaN,NaN,1.000000,150.000000,NaN,NaN,10.000000,150.000000,NaN,19.000000,NaN


## 3. Missing Values Check

Missing data can break aggregations and BI measures.
We verify completeness across all columns.


In [32]:
missing_summary = df.isna().sum().to_frame("missing_count")
missing_summary["missing_pct"] = missing_summary["missing_count"] / len(df)

missing_summary


,missing_count,missing_pct
Unnamed: 0,0,0.0
Order ID,0,0.0
Product Category,0,0.0
Product,0,0.0
Quantity Ordered,0,0.0
Price Each,0,0.0
Order Date,0,0.0
Purchase Address,0,0.0
Month,0,0.0
Sales,0,0.0


## 4. Business Rule Validation

We validate core transactional logic:
- Quantity must be positive
- Price must be positive
- Sales must equal Quantity × Price


In [33]:
df["calculated_sales"] = df["Quantity Ordered"] * df["Price Each"]

df["valid_quantity"] = df["Quantity Ordered"] > 0
df["valid_price"] = df["Price Each"] > 0
df["valid_sales"] = np.isclose(
    df["Sales"],
    df["calculated_sales"],
    atol=0.01
)


### Validation Results (% of records passing)


In [34]:
validation_results = pd.DataFrame({
    "quantity_valid_pct": [df["valid_quantity"].mean()],
    "price_valid_pct": [df["valid_price"].mean()],
    "sales_match_pct": [df["valid_sales"].mean()]
})

validation_results


,quantity_valid_pct,price_valid_pct,sales_match_pct
0,1.0,1.0,1.0


## 5. Order ID Duplication Analysis

Order IDs are expected to repeat because
one order can contain multiple products.

We confirm this assumption explicitly.


In [35]:
order_counts = df["Order ID"].value_counts()

(order_counts > 1).mean()


np.float64(0.039991705756093184)

## 6. Revenue Reconciliation

We validate total revenue consistency between:
- Reported sales
- Calculated sales


In [36]:
reported_revenue = df["Sales"].sum()
calculated_revenue = df["calculated_sales"].sum()

pd.DataFrame({
    "reported_revenue": [reported_revenue],
    "calculated_revenue": [calculated_revenue],
    "difference": [reported_revenue - calculated_revenue]
})


,reported_revenue,calculated_revenue,difference
0,34492035.97,34492035.97,0.0


## 7. Temporal Coverage

We verify that the dataset spans expected months and hours.
This prevents incomplete or truncated reporting.



In [37]:
df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")

df["Order Date"].min(), df["Order Date"].max()


C:\Users\Envy\AppData\Local\Temp\ipykernel_19788\2047891586.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")


(Timestamp('2019-01-01 03:07:00'), Timestamp('2020-01-01 05:13:00'))

In [38]:
df["Month"].value_counts().sort_index()


Month
1      9709
2     11975
3     15153
4     18279
5     16566
6     13554
7     14293
8     11961
9     11621
10    20282
11    17573
12    24984
Name: count, dtype: int64

In [39]:
df["Hour"].value_counts().sort_index()


Hour
0      3910
1      2350
2      1243
3       831
4       854
5      1321
6      2482
7      4011
8      6256
9      8748
10    10944
11    12411
12    12587
13    12129
14    10984
15    10175
16    10384
17    10899
18    12280
19    12905
20    12228
21    10921
22     8822
23     6275
Name: count, dtype: int64

## 8. Categorical Coverage

We ensure product categories and cities are populated
and reasonably distributed.


In [40]:
df["Product Category"].value_counts(normalize=True)


Product Category
Audio Devices             0.256822
Charging Cables           0.234262
Batterie                  0.221662
Monitors                  0.129169
Phones and Accessories    0.077612
Laptops and Computers     0.047604
Entertainment Devices     0.025813
Home Appliances           0.007056
Name: proportion, dtype: float64

In [41]:
df["City"].nunique()


9

## 9. Final Data Quality Assessment

### Summary
- Dataset is structurally complete
- Business rules largely satisfied
- Revenue calculations reconcile
- Time and categorical coverage are valid

### Conclusion
The dataset is suitable for downstream normalization,
SQL modeling, and Power BI analytics.
